<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
# Импорт основных библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Импорт библиотек для работы с текстами и NLP
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Импорт инструментов машинного обучения
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

# Импорт векторизаторов текста
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Загрузка ресурсов NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Инициализация стоп-слов
stop_words = stopwords.words('english')

# Настройка отображения данных
pd.set_option('display.max_colwidth', 1000)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Подготовка

In [2]:
# Загрузка данных из CSV-файла
df = pd.read_csv('/datasets/toxic_comments.csv')

# Подсветим токсичные комментарии
def highlight_toxic(row):
    color = 'background-color: lightcoral' if row['toxic'] == 1 else ''
    return [color] * len(row)

# Применим стиль и выведем первые строки
display(df.head(10).style.apply(highlight_toxic, axis=1))

,Unnamed: 0,text,toxic
0,0,"Explanation Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,""" More I can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know. There appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0
5,5,""" Congratulations from me as well, use the tools well. · talk """,0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0
8,8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0
9,9,alignment on this subject and which are contrary to those of DuLithgow,0


In [3]:
# Проверим размер таблицы
print(f"Размер таблицы: {df.shape}")

Размер таблицы: (159292, 3)


In [4]:
# Вывод информации о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


У нас 159 292 комментария в датасете.


Столбец text содержит тексты комментариев.


Столбец toxic содержит метку (0 — нормальный комментарий, 1 — токсичный).


Пропусков в данных нет.

In [5]:
# Проверка количества дубликатов
print(f"Количество дубликатов: {df.duplicated().sum()}")

Количество дубликатов: 0


## Обучение

In [6]:
def get_wordnet_pos(tag):
    """
    Переводит POS-тег (NN, VB, JJ и т.д.) из NLTK
    в формат, который понимает WordNetLemmatizer (wordnet.NOUN и т.п.).
    """
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [7]:
class ToxicCommentClassifier:
    """Класс для предобработки текстов и классификации комментариев."""

    def __init__(self, models_and_params, score, solvers, stop_words, df, target_column, text_column):
        """
        Инициализация класса.
        
        Аргументы:
        - models_and_params: список кортежей (модель, параметры для GridSearchCV)
        - score: метрика для оценки
        - solvers: список векторизаторов (например, [TfidfVectorizer, CountVectorizer])
        - stop_words: список стоп-слов
        - df: DataFrame с данными
        - target_column: название целевого столбца
        - text_column: название столбца с текстом
        """
        self.models_and_params = models_and_params
        self.score = score
        self.solvers = solvers
        self.stop_words = stop_words
        self.df = df
        self.target_column = target_column
        self.text_column = text_column

        # Проверка наличия нужных столбцов
        if target_column not in df.columns or text_column not in df.columns:
            raise ValueError("Ошибка: Указанные столбцы не найдены в DataFrame!")

        # Переменные для хранения данных
        self.best_model = None
        self.max_score = -1
        self.results = {}

        # Запуск этапов предобработки
        self._clean_text()
        print("✅ Текст очищен.")
        self._lemmatize()
        print("✅ Лемматизация завершена.")
        self._split_data()
        print("✅ Данные разделены на train/val/test.")

    def _clean_text(self):
        """Очищает текст от лишних символов."""
        self.df[self.text_column] = self.df[self.text_column].astype(str).str.lower()
        self.df[self.text_column] = self.df[self.text_column].apply(lambda x: re.sub(r'[^a-z\s]', '', x))
        self.df[self.text_column] = self.df[self.text_column].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

    def _lemmatize(self):
        """Лемматизирует текст с учётом частей речи (POS-тегов)."""
        lemmatizer = WordNetLemmatizer()

        def lemmatize_text(text):
            tokens = nltk.word_tokenize(text)
            pos_tags = nltk.pos_tag(tokens)
            return " ".join([
                lemmatizer.lemmatize(token, get_wordnet_pos(pos))
                for token, pos in pos_tags
            ])
        self.df[self.text_column] = self.df[self.text_column].apply(lemmatize_text)

    def _split_data(self):
        """Разделяет данные на train, validation и test."""
        X_train, X_test, y_train, y_test = train_test_split(
            self.df[self.text_column], 
            self.df[self.target_column], 
            test_size=0.2, 
            random_state=42
        )
        X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
        self.splits = {
            'train': (X_train, y_train),
            'val': (X_val, y_val),
            'test': (X_test, y_test)
        }


    def train_models(self):
        """Обучает модели с использованием Pipeline, чтобы избежать утечки данных."""
        self.results = {}

        for model, params in tqdm(self.models_and_params, desc="Обучение моделей"):
            for vectorizer in self.solvers:
                
                pipe = Pipeline([
                    ('vect', vectorizer(stop_words=self.stop_words)),
                    ('clf', model)
                ])
            
            
                param_distributions = {f'clf__{key}': val for key, val in params.items()}

            
                rand = RandomizedSearchCV(
                    estimator=pipe,
                    param_distributions=param_distributions,
                    n_iter=2,              
                    cv=3,
                    scoring=self.score,
                    n_jobs=-1,
                    random_state=42
                )

                X_train, y_train = self.splits['train']
                X_val, y_val = self.splits['val']
                X_test, y_test = self.splits['test']

                rand.fit(X_train, y_train)

                best_model = rand.best_estimator_
                best_score_train = rand.best_score_
                best_score_val = f1_score(y_val, best_model.predict(X_val))
                best_score_test = f1_score(y_test, best_model.predict(X_test))

                name = f"{model.__class__.__name__} + {vectorizer.__name__}"
                self.results[name] = {
                    "best_model": best_model,
                    "train_score": best_score_train,
                    "val_score": best_score_val,
                    "test_score": best_score_test
                }

                if best_score_val > self.max_score:
                    self.max_score = best_score_test
                    self.best_model = best_model

        return {"max_score": self.max_score, "best_model": self.best_model}


    def get_results(self):
        """Возвращает словарь с результатами обучения."""
        return self.results

In [8]:
# Берём 20% данных
df_small = df.sample(frac=0.2, random_state=42).reset_index(drop=True)

# Гиперпараметры для моделей
params_Log = {
    "max_iter": [1000],
    "C": [1, 5]
}

params_RF = {
    'n_estimators': [50],
    'max_depth': [10]
}

In [9]:
# Создаём объект классификатора

cl_small = ToxicCommentClassifier(
    models_and_params=[
        (LogisticRegression(random_state=42, class_weight='balanced', n_jobs=-1), params_Log),
        (RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1), params_RF)
    ],
    score='f1',
    solvers=[TfidfVectorizer, CountVectorizer],
    stop_words=stop_words,
    df=df_small,
    target_column='toxic',
    text_column='text'
)

✅ Текст очищен.
✅ Лемматизация завершена.
✅ Данные разделены на train/val/test.


In [10]:
cl_small.df.head(5)

,Unnamed: 0,text,toxic
0,31055,sometime back i just happen to log on to wwwizoomin with a friend reference and i be amaze to see the concept fresh idea entertainment have come up with so many deal all under one roof this website be very user friendly and easy to use and be fun to be on you have gossip game facts another exciting feature to add to it be face of the week every week new face be select and put up a izoom face it great to have be select in four out of a group of million this new start up have already get many a deal in it kitty few of them be thefortune hotel the beach be my personal favorite izoomin have a usp of mobile coupon coupon be available even when a user can not access internet you just need to sms izoom support to and you get attend immediately all i can say be izoomin be a must visit website for everyone before they go out for shopping or dining or for out cheer,0
1,102929,the late edit be much well dont make this article state super at all,0
2,67385,october utc i would think youd be able to get your point across and be immune to any objection be you to simply embellish the second sentence of the article by change he be school at thornleigh salesian college to he be school at the then allcatholic thornleigh salesian college good suggestion from an anon what do you think rgds,0
3,81167,thanks for the tip on the currency translation think it all do now,0
4,90182,i would argue that if content on the con in comparison to the art music be out of proportion then it warrant far contribution to the article not the removal of an indepth piece of content also a i mention before the art music unit have a notable history comparable to that of the con itself because of this i would far argue that content on the art music unit be more relevant to this article than the information on the newcastle conservatorium,0


In [11]:
# Запуск обучения
results_small = cl_small.train_models()

# Выводим результаты
print("Результаты обучения на 20% данных:", results_small)
print("Лучшая модель:", cl_small.best_model)
print("Лучший тестовый F1:", cl_small.max_score)

Обучение моделей:  50%|█████     | 1/2 [07:21<07:21, 441.73s/it]/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 1 is smaller than n_iter=2. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 1 is smaller than n_iter=2. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
Обучение моделей: 100%|██████████| 2/2 [07:32<00:00, 226.47s/it]

Результаты обучения на 20% данных: {'max_score': 0.7503828483920368, 'best_model': Pipeline(steps=[('vect',
                 CountVectorizer(stop_words=['a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...])),
                ('clf',
                 LogisticRegression(C=5, class_weight='balanced', max_iter=1000,
                                    n_jobs=-1, random_state=42))])}
Лучшая модель: Pipeline(steps=[('vect',
                 CountVectorizer(stop_words=['a', 'about', 'above', 

## Выводы

1. По результатам обучения на 20% данных:


- Лучшая модель: LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1, random_state=42)


- Лучшая метрика F1: ≈ 0.75


2. Поскольку модель достигла F1 выше 0.75 даже на уменьшенной выборке, можно сделать вывод, что:


- Логистическая регрессия с учётом баланса классов неплохо справляется с задачей определения токсичности комментариев.


- Для ускорения расчётов мы использовали 20% данных и сократили сетку гиперпараметров; при обучении на полном датасете модель, вероятно, может показать ещё более стабильный результат, но потребует больше времени.


Таким образом, проект успешно демонстрирует построение модели классификации токсичных комментариев с требуемым уровнем качества F1 ≥ 0.75.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны